In [47]:
%cd /content/drive/MyDrive/projects/SMS SPAM CLASSIFIER

/content/drive/MyDrive/projects/SMS SPAM CLASSIFIER


In [48]:
#RUN TO DOWNLOAD DEPENDENCIES
import joblib
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:
import pandas as pd

In [50]:
training_data = pd.read_csv("/content/drive/MyDrive/projects/SMS SPAM CLASSIFIER/training_data.csv")

In [51]:
training_data.head(10)

,msg_text,ham_or_spam
0,"Haha better late than ever, any way I could sw...",ham
1,Thanks a lot for your wishes on my birthday. T...,ham
2,A guy who gets used but is too dumb to realize...,ham
3,Talk sexy!! Make new friends or fall in love i...,spam
4,Nowadays people are notixiquating the laxinorf...,ham
5,PRIVATE! Your 2004 Account Statement for 07742...,spam
6,Gimme a few was &lt;#&gt; minutes ago,ham
7,K sure am in my relatives home. Sms me de. Pls:-),ham
8,Lol wtf random. Btw is that your lunch break,ham
9,V skint too but fancied few bevies.waz gona go...,ham


In [52]:
# convert the DataFrame to a NumPy array

data = training_data.to_numpy()
data

array([['Haha better late than ever, any way I could swing by?', 'ham'],
       ['Thanks a lot for your wishes on my birthday. Thanks you for making my birthday truly memorable.',
        'ham'],
       ['A guy who gets used but is too dumb to realize it.', 'ham'],
       ...,
       ["Prabha..i'm soryda..realy..frm heart i'm sory", 'ham'],
       ['Nt joking seriously i told', 'ham'],
       ['Did he just say somebody is named tampa', 'ham']], dtype=object)

In [53]:
X = data[:, 0]
y = data[:, 1]
X.shape, y.shape


((5014,), (5014,))

# Tokenization

In [54]:
# With the help of NLTK tokenize.regexp() module, we are able to extract the tokens from string by using regular expression with RegexpTokenizer() method.

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer('\w+')

# Stopwords

In [55]:
from nltk.corpus import stopwords
sw = set(stopwords.words('english'))

# Tokenizing -> Stopwords Removal -> Stemming


In [56]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

def stemming(sentence):
    sentence = sentence.lower()
    tokens = tokenizer.tokenize(sentence) 
    removed_stopwords = [w for w in tokens if w not in sw]
    stemmed_words = [ps.stem(token) for token in removed_stopwords]
    clean_sentence = ' '.join(stemmed_words)
    return clean_sentence

In [57]:
# GET A CLEAN DATASET
def getClean(document):
    d = []
    for doc in document:
        d.append(stemming(doc))
    return d

In [58]:
stemmed_doc = getClean(X)

# Count Vectorization

CountVectorizer means breaking down a sentence or any text into words by performing preprocessing tasks like converting all words to lowercase, thus removing special characters.


In [59]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [60]:
vc = cv.fit_transform(stemmed_doc)
X = vc.todense()

# Splitting into training and testing data to calculate model accuracy


In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Implementing Naive Bayes' Algorithm

**Why We Using Naive Bayes as an Algorithms for Filtering the Email:-**

Naive Bayes work on dependent events and the probability of an event occurring in the future that can be detected from the previous occurring of the same event . This technique can be used to classify spam e-mails, words probabilities play the main rule here. If some words occur often in spam but not in ham, then this incoming e-mail is probably spam. Naive Bayes classifier technique has become a very popular method in mail filtering Email. Every word has certain probability of occurring in spam or ham email in its database. If the total of words probabilities exceeds a certain limit, the filter will mark the e-mail to either category. Here, only two categories are necessary: spam or ham.

In [63]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train, y_train)
print(y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


['ham' 'ham' 'ham' ... 'ham' 'ham' 'spam']


# Model Accuracy


In [64]:
model.score(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


0.9840478564307079

# Prediction on Test Data


In [65]:
test_data = pd.read_csv('test_data.csv')

test = test_data.to_numpy()
test_messages = test[:, 0]
type(test)
test_messages.shape

(558,)

In [66]:
def prepare(messages):
    d = getClean(messages)
    return cv.transform(d)

In [67]:
messages = prepare(test_messages)
y_pred = model.predict(messages)
test_data['ham_or_spam'] = y_pred
test_data

,msg_text,ham_or_spam
0,Squeeeeeze!! This is christmas hug.. If u lik ...,ham
1,And also I've sorta blown him off a couple tim...,ham
2,Mmm thats better now i got a roast down me! i...,ham
3,Mm have some kanji dont eat anything heavy ok,ham
4,So there's a ring that comes with the guys cos...,ham
...,...,...
553,Heart is empty without love.. Mind is empty wi...,ham
554,"Alright we'll bring it to you, see you in like...",ham
555,If You mean the website. Yes.,ham
556,"Ya they are well and fine., BBD(pooja) full pi...",ham


# Exporting to csv

In [68]:
test_data.to_csv('Predicted.csv')